# Team 5 💣
1. Maximus Lee
2. Aloysius Woo
3. Lim Huai Fu
4. Tan Ai Xin
5. Jin Zhenglong

## Setup

### **Install Environment**
Est Time: 8 mins
- Python Environment
- GitHub Repo
- Folder Scaffolding
- FYP/MMPose Scaffolding

In [2]:
#@title Python

def setup_python():
  %cd /content
  !sudo apt-get install python3.8
  !sudo apt-get install python3.8-distutils

  !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
  !apt-get update
  !apt install software-properties-common
  !sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
  !apt-get install python3-pip
  !apt-get install libcairo2-dev
  !apt update

In [3]:
#@title GitHub Repos

def setup_github():
  # Empty default folder
  %cd /content

  !git clone https://github.com/maximus-lee-678/ict3104_team_05.git .

setup_github()

/content
fatal: destination path '.' already exists and is not an empty directory.


In [4]:
#@title FollowYourPose/MMPose Folder Scaffolding

def setup_folder_scaffolding():
  # FollowYourPose
  %cd /content/FollowYourPose
  %mkdir /content/FollowYourPose/checkpoints

In [5]:
#@title FollowYourPose/MMPose Libaries
# https://github.com/open-mmlab/mmpose
# https://blog.csdn.net/qq_21532607/article/details/130226728
# https://colab.research.google.com/github/open-mmlab/mmpose/blob/master/demo/MMPose_Tutorial.ipynb

def setup_dependency_lib():
  %cd /content
  !python3.8 -m pip install -r other_files/requirements/requirements_colab.txt

  !python3.8 -m mim install mmengine
  !python3.8 -m mim install mmpose
  !python3.8 -m mim install "mmcv>=2.0.0"
  !python3.8 -m mim install "mmdet>=3.0.0"

  !python3.8 -m pip install --upgrade bitsandbytes
  !python3.8 -m pip install --upgrade omegaconf


### **Drive Mounting**
Mount files from google drive
- checkpoints

In [6]:
# @title Google Account Selection
def mount_account_select():
  %cd /content
  !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !sudo apt-get update -qq 2>&1 > /dev/null
  !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
  !google-drive-ocamlfuse

In [7]:
# @title Mount Drive Contents

def mount_drive():
  !sudo apt-get install -qq w3m # to act as web browser
  !xdg-settings set default-web-browser w3m.desktop # to set default browser
  %cd /content
  !mkdir gdrive
  %cd gdrive
  !mkdir MyDrive
  %cd ..
  %cd ..
  !google-drive-ocamlfuse /content/gdrive/MyDrive

### **Project Dir Setup**

In [8]:
# @title Clear sample files { display-mode: "form" }

!find /content -mindepth 1 -delete

In [9]:
# @title Init Path { display-mode: "form" }

# Root Directory
ROOT_DIR_PATH = "/content"

# Video
VIDEO_DIR_PATH = f"{ROOT_DIR_PATH}/video"
VIDEO_SKELETON_DIR_PATH = f"{ROOT_DIR_PATH}/video/Skeleton"

# I/O Files
TRAINING_CONTENT_DIR_PATH = f"{ROOT_DIR_PATH}/training_content"
CUSTOM_MODEL_DIR_PATH = f"{ROOT_DIR_PATH}/custom_model"
INFERENCE_OUTPUT_DIR_PATH = f"{ROOT_DIR_PATH}/inference_output"

MMPOSE_DIR_PATH = f"{ROOT_DIR_PATH}/MMPose"
FYP_DIR_PATH = f"{ROOT_DIR_PATH}/FollowYourPose"

# FYP Config
CONFIG_DIR_PATH = f"{FYP_DIR_PATH}/configs"

def create_dir_path():
  %mkdir {VIDEO_DIR_PATH}
  %mkdir {VIDEO_SKELETON_DIR_PATH}
  %mkdir {TRAINING_CONTENT_DIR_PATH}
  %mkdir {CUSTOM_MODEL_DIR_PATH}
  %mkdir {INFERENCE_OUTPUT_DIR_PATH}
  %mkdir {CONFIG_DIR_PATH}

In [71]:
# @title Pre-setup { display-mode: "form" }

import sys

# Check if the path is not already in sys.path before appending
path_to_append = '/usr/local/lib/python3.8/dist-packages'
if path_to_append not in sys.path:
    sys.path.append(path_to_append)

import os
import csv
import datetime
from enum import Enum
import psutil
import yaml

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Video, clear_output


In [11]:
# @title Symbolic Linking { display-mode: "form" }

def symbolic_linking():
  # Video selection
  %mkdir /content/video/
  !ln -s /content/gdrive/MyDrive/ICT3104/video/* /content/video

  # Checkpoints folder
  !ln -s /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints

  # Charade Lookup Files
  %mkdir /content/charades_lookup/
  !ln -s /content/gdrive/MyDrive/ICT3104/charades_lookup/* /content/charades_lookup

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/video/* /content/video/
# %cp -r /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints/

## Video Browsing

In [12]:
# @title Video Selection { display-mode: "form" }

# Helper Functions
def getFolderContent(folder_name):
  subfolders = []
  for content in os.listdir(VIDEO_DIR_PATH):
        content_path = os.path.join(VIDEO_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            subfolders.append(content)

  return subfolders

class VideoTab:

  def __init__(self, state):
    # Init
    self.state = state
    self.video_dir_folders = os.listdir(VIDEO_DIR_PATH)
    self.video_dir_folder_content = getFolderContent(self.video_dir_folders)

    # Create layout
    self.layout_single_button = widgets.Layout(width='212px',margin='3px 0px 0px 90px')
    self.layout_double_button = widgets.Layout(width='104px')
    self.layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
    self.layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start', height="532px")

    # Create widgets
    self.video_output_placeholder = widgets.Output(layout=self.layout_output)
    self.video_subdir_dropdown = widgets.Dropdown(options=self.video_dir_folder_content, description='Folder:', value=None, disabled=False)
    self.video_dropdown = widgets.Dropdown(options=[], description='Video:', disabled=True)
    self.video_display_button = widgets.Button(description="Display", disabled=True, layout=self.layout_single_button)
    self.video_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=self.layout_single_button)
    self.video_mmpose_button = widgets.Button(description="MMPose", disabled=True, layout=self.layout_single_button)
    self.video_fyp_button = widgets.Button(description="FYP", disabled=True, layout=self.layout_single_button)

    self.video_hbox_1 = widgets.HBox([self.video_display_button, self.video_refresh_button], layout=self.layout_hbox)
    self.video_hbox_2 = widgets.HBox([self.video_mmpose_button, self.video_fyp_button], layout=self.layout_hbox)

    self.video_output_placeholder_content = widgets.HTML("""
      <div style="width: 512px; height: 512px; border-radius: 5%; background-color: black; margin: 0 auto; display: flex; justify-content: center; align-items: center;">
          <div style="width: 500px; height: 500px; border-radius: 5%; border: 2px solid white;" />
      </div>
    """)

  # Create listeners
  ## Update video dropdown options based on the selected folder
  def video_subdir_select(self, change):
      selected_video_folder = self.video_subdir_dropdown.value

      if selected_video_folder != None:
        selected_VIDEO_DIR_PATH = f"{VIDEO_DIR_PATH}/{selected_video_folder}"
        selected_video_dir_content = [file for file in os.listdir(selected_VIDEO_DIR_PATH) if file.endswith('.mp4')]
      else:
        selected_video_dir_content = []

      self.video_dropdown.options = selected_video_dir_content
      if not selected_video_dir_content:
          self.video_dropdown.disabled = False
          self.video_dropdown.value = None
      else:
          self.video_dropdown.disabled = False

  ## Display the selected video
  def display_selected_video(self, change):
      selected_video = self.video_dropdown.value
      selected_video_folder = self.video_subdir_dropdown.value

      if selected_video:
          video_path = f"{VIDEO_DIR_PATH}/{selected_video_folder}/{selected_video}"
          video_display = Video(video_path, width=512, height=512, embed=True)

          # Clear the output placeholder and display the video
          with self.video_output_placeholder:
              clear_output(wait=True)
              display(video_display)

  ## Refresh folder and directory
  def refresh_folder_and_directory(self, change):
      video_dir_folders = os.listdir(VIDEO_DIR_PATH)
      video_dir_folder_content = getFolderContent(video_dir_folders)

      self.video_dropdown.options = []
      self.video_dropdown.value = None

      self.video_subdir_dropdown.options = video_dir_folder_content
      self.video_subdir_dropdown.value = None

  ## Enable button when a valid video is picked
  def enable_button(self, change):
      if self.video_dropdown.value:
          self.video_display_button.disabled = False
          self.video_mmpose_button.disabled = False
          self.video_fyp_button.disabled = False
      else:
          self.video_display_button.disabled = True
          self.video_mmpose_button.disabled = True
          self.video_fyp_button.disabled = True

  def change_mmpose_state(self, change):
    content = [
        self.video_subdir_dropdown.value,
        self.video_dropdown.value
    ]
    self.state.change_state(State.MMPOSE, content)

  def change_fyp_state(self, change):
    content = [
        self.video_subdir_dropdown.value,
        self.video_dropdown.value
    ]
    self.state.change_state(State.FYP, content)

  def attach_listeners(self):
    # Attach Listeners
    self.video_subdir_dropdown.observe(self.video_subdir_select, 'value')
    self.video_display_button.on_click(self.display_selected_video)
    self.video_refresh_button.on_click(self.refresh_folder_and_directory)
    self.video_dropdown.observe(self.enable_button, 'value')
    self.video_mmpose_button.on_click(self.change_mmpose_state)
    self.video_fyp_button.on_click(self.change_fyp_state)

  def display_content(self):
    # Display fields
    hr = widgets.HTML("<hr>", layout=widgets.Layout(margin="3px 0px 0px 90px"))
    with self.video_output_placeholder:
        display(self.video_output_placeholder_content)

    with self.state.video_output:
      pane = widgets.VBox([
        self.video_subdir_dropdown,
        self.video_dropdown,
        self.video_refresh_button,
        self.video_display_button,
        hr,
        self.video_mmpose_button,
        self.video_fyp_button
      ])
      happy_box = widgets.HBox([self.video_output_placeholder, pane], layout=widgets.Layout(margin="10px 0px 10px 0px"))
      display(happy_box)




## Inference: MMPOSE

In [13]:
# @title Settings { display-mode: "form" }

# Helper Functions
## Re-encode video due to H.264 video encoding error
def reencode_video(video_name):
  input_file = f"{VIDEO_SKELETON_DIR_PATH}/{video_name}"
  temp_output_file = f"{VIDEO_SKELETON_DIR_PATH}/output.mp4"

  !ffmpeg -i {input_file} -c:v libx264 -crf 23 -c:a aac -strict experimental {temp_output_file}

  os.remove(input_file)
  os.rename(temp_output_file, input_file)

def run_mmpose_inference(folder_path, video_name):
    full_path = f"{VIDEO_DIR_PATH}/{folder_path}/{video_name}"

    print(full_path)
    %cd -q {MMPOSE_DIR_PATH}

    if not os.path.exists(VIDEO_SKELETON_DIR_PATH):
        os.mkdir(VIDEO_SKELETON_DIR_PATH)

    # Start inference
    !python demo/inferencer_demo.py \
        {full_path}  \
        --pose2d human --vis-out-dir {VIDEO_SKELETON_DIR_PATH} \
        --black-background


## Inference: FYP

In [72]:
# @title Config { display-mode: "form" }


class FYPConfig:
  # Init

  def __init__(self):

    from omegaconf import OmegaConf
    # Const
    self.OPTIONS = ["- Select an Item -", "True", "False"]
    self.SAMPLE_YAML_PATH = f"{CONFIG_DIR_PATH}/pose_sample.yaml"

    %cd -q {FYP_DIR_PATH}
    load_config = OmegaConf.load(self.SAMPLE_YAML_PATH)
    self.load_config = load_config

    model_options = [("Default", f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4")]
    for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
        model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
        if os.path.isdir(model_path):
            model_options.append((model_name, f"{CUSTOM_MODEL_DIR_PATH}/{model_name}"))

    output_folder = load_config.output_dir.split("/")

    # Create layout
    config_style = {'description_width': '150px'}
    config_layout = widgets.Layout(width="500px")
    config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')

    # Inference Data
    self.pretrained_model_path = widgets.Dropdown(options=model_options,description="pretrained_model_path:",value=load_config.pretrained_model_path,layout=config_layout,style=config_style)
    self.output_dir = widgets.Text(description="output_folder_name:", value=output_folder[3] if len(output_folder) == 4 else "", style=config_style, layout=config_layout)
    self.train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config.train_batch_size, style=config_style, layout=config_layout)
    self.validation_steps = widgets.IntText(description="validation_steps:", value=load_config.validation_steps, style=config_style, layout=config_layout)
    self.seed = widgets.IntText(description="seed:", value=load_config.seed, style=config_style, layout=config_layout)
    self.mixed_precision = widgets.Text(description="mixed_precision:", value=load_config.mixed_precision, style=config_style, layout=config_layout)
    self.gradient_checkpointing = widgets.Dropdown(options=self.OPTIONS, value="True" if load_config.gradient_checkpointing else "False", description="gradient_checkpointing:", layout= config_layout, style=config_style)
    self.enable_xformers_memory_efficient_attention = widgets.Dropdown(options=self.OPTIONS, value="True" if load_config.enable_xformers_memory_efficient_attention else "False", description="enable_xformers_memory_efficient_attention:", layout= config_layout, style=config_style)

    # Validation Data
    load_validation = load_config.validation_data
    self.prompts = widgets.Textarea(description="prompts:", value="\n".join(load_validation.prompts), style=config_style, layout=widgets.Layout(width="500px", height="100px"))
    self.video_length = widgets.IntText(description="video_length:", value=load_config.validation_data.video_length, style=config_style, layout=config_layout)
    self.width = widgets.IntText(description="width:", value=load_config.validation_data.width, style=config_style, layout=config_layout)
    self.height = widgets.IntText(description="height:", value=load_config.validation_data.height, style=config_style, layout=config_layout)
    self.num_inference_steps = widgets.IntText(description="num_inference_steps:", value=load_config.validation_data.num_inference_steps, style=config_style, layout=config_layout)
    self.guidance_scale = widgets.FloatText(description="guidance_scale:", value=load_config.validation_data.guidance_scale, style=config_style, layout=config_layout)
    self.use_inv_latent = widgets.Dropdown(options=self.OPTIONS, value="True" if load_config.validation_data.use_inv_latent else "False", description="use_inv_latent:", layout= config_layout, style=config_style)
    self.num_inv_steps = widgets.IntText(description="num_inv_steps:", value=load_config.validation_data.num_inv_steps, style=config_style, layout=config_layout)
    self.num_dataset_set = widgets.Text(description="dataset_set:", value=load_config.validation_data.dataset_set, style=config_style, layout=config_layout)

    # Subheader
    self.header_inference = widgets.HTML(value="<h3>Inference Data</h3>")
    self.header_validation = widgets.HTML(value="<h3>Validation Data</h3>")

  def getForm(self):
    config_vbox = widgets.VBox([
        self.header_inference,
        self.output_dir,
        self.pretrained_model_path,
        self.train_batch_size,
        self.validation_steps,
        self.seed,
        self.mixed_precision,
        self.gradient_checkpointing,
        self.enable_xformers_memory_efficient_attention,

        self.header_validation,
        self.prompts,
        self.video_length,
        self.width,
        self.height,
        self.num_inference_steps,
        self.guidance_scale,
        self.num_inv_steps,
        self.num_dataset_set
    ])

    return config_vbox

  # Create listeners
  def save_config(self):
    from omegaconf import OmegaConf

    config = {
        "pretrained_model_path": self.pretrained_model_path.value,
        "output_dir": f"{INFERENCE_OUTPUT_DIR_PATH}/{self.output_dir.value}",
        "validation_data": {
            "prompts": [prompt.strip() for prompt in self.prompts.value.splitlines() if prompt.strip()],
            "video_length": self.video_length.value,
            "width": self.width.value,
            "height": self.height.value,
            "num_inference_steps": self.num_inference_steps.value,
            "guidance_scale": self.guidance_scale.value,
            "use_inv_latent": self.use_inv_latent.value,
            "num_inv_steps": self.num_inv_steps.value,
            "dataset_set": self.num_dataset_set.value
        },
        "train_batch_size": self.train_batch_size.value,
        "validation_steps": self.validation_steps.value,
        "resume_from_checkpoint": self.load_config.resume_from_checkpoint,
        "seed": self.seed.value,
        "mixed_precision": self.mixed_precision.value,
        "gradient_checkpointing": self.gradient_checkpointing.value,
        "enable_xformers_memory_efficient_attention": self.enable_xformers_memory_efficient_attention.value
    }
    with open(self.SAMPLE_YAML_PATH, "w") as file:
      yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)


In [15]:
# @title Settings { display-mode: "form" }

# Create layout
layout_single_long_button = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
selected_fyp_video_input = widgets.Text(placeholder='Select a video above', description="Video:", disabled=True)
config_file_path = widgets.Text(value='configs/pose_sample.yaml',description="Config File:")
inf_fyp_button = widgets.Button(description="Start Inference", disabled=True, layout=layout_single_long_button)

# Create listeners
## Retrieve video input
def update_fyp_input_video(change):
    selected_video = video_dropdown.value
    selected_fyp_video_input.value = f"../video/{video_subdir_dropdown.value}/{selected_video}"
    if selected_video:
        inf_fyp_button.disabled = False
    else:
        inf_fyp_button.disabled = True

## Run FYP
def run_fyp_inference(button):
    config_file_path_text = config_file_path.value
    video_file_path_text = selected_fyp_video_input.value

    %cd {FYP_DIR_PATH}

    # Start inference
    !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py \
        --config={config_file_path_text}  \
        --skeleton_path={video_file_path_text}

# Attach Listeners
inf_fyp_button.on_click(run_fyp_inference)
video_fyp_button.on_click(update_fyp_input_video)

# Display fields
display(selected_fyp_video_input)
display(config_file_path)
display(inf_fyp_button)

NameError: ignored

In [15]:
def run_fyp_inference(config_file_path, video_file_path):
    %cd {FYP_DIR_PATH}

    # Start inference
    !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py \
        --config={config_file_path}  \
        --skeleton_path={video_file_path}

## Training (WIP)

In [ ]:
# @title Init { display-mode: "form" }

# Charades Data Class from csv
class CharadesData:
  def __init__(self, row):
    id, subject, scene, quality, relevance, verified, script, objects, descriptions, actions, length = row
    self.id = id
    self.subject = subject
    self.scene = scene
    self.quality = quality
    self.relevance = relevance
    self.verified = verified
    self.script = script
    self.objects = objects.split(";")
    self.descriptions = descriptions
    self.length = length
    self.actions = {}

    # Convert actions in proper data structure ("class_id time_start time_end" -> class_id: [time_start, time_end])
    if len(actions) != 0:
      action_substrings = actions.split(';')
      for substring in action_substrings:
        parts = substring.split()
        key = parts[0]
        values = [self.convert_to_ms(parts[1]), self.convert_to_ms(parts[2])]
        self.actions[key] = values

  # For printing
  def __str__(self):
        return f"ID: {self.id}, Subject: {self.subject}, Scene: {self.scene}, Quality: {self.quality}, Relevance: {self.relevance}, Verified: {self.verified}, Script: {self.script}, Objects: {self.objects}, Descriptions: {self.descriptions}, Actions: {self.actions}, Length: {self.length}"

  # Helper function to convert time into ms
  def convert_to_ms(self, seconds):
    ss,ms = seconds.split('.')
    total_ms = 1000*int(ss) + int(ms)
    return total_ms

  # Caption getter with template
  def getCaption(self, index):
    return f"In a {self.scene} setting, within the context of '{self.script}', the action '{action_descriptions[list(self.actions.keys())[index]]}' is taking place."


action_descriptions = {}
charades_all = []

# Load classes lookup table
with open('/content/charades_lookup/Charades_v1_classes.txt', 'r') as file:
    for line in file:
        code, description = line.strip().split(' ', 1)
        action_descriptions[code] = description

# Load charades data A
with open('/content/charades_lookup/Charades_v1_train.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)

    for row in csv_reader:
        charadeData = CharadesData(row)
        charades_all.append(charadeData)

# Load charades data B
with open('/content/charades_lookup/Charades_v1_test.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)

    for row in csv_reader:
        charadeData = CharadesData(row)
        charades_all.append(charadeData)

clear_output()
print("Data load successful!")

FileNotFoundError: ignored

In [ ]:
# @title Dataset Preload (Video) { display-mode: "form" }

# Init
dataset_dir_folders = os.listdir(VIDEO_DIR_PATH)

## Training env
training_dataset = None
training_idx = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
training_branch = f"{TRAINING_CONTENT_DIR_PATH}/{training_idx}"

# Create layout
config_style = {'description_width': '100px'}
config_layout = widgets.Layout(width="300px")
config_button_layout = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
dataset_folder_dropdown = widgets.Dropdown(options=[dir for dir in dataset_dir_folders if not dir.startswith(".")], description='Dataset Folder:', value=None, layout=config_layout, style=config_style)
dataset_cutting_button = widgets.Button(description="Start Cutting", disabled=True, layout=config_button_layout)

# Create listeners
def dataset_dir_select(change):
  if dataset_folder_dropdown:
    dataset_folder_content = os.listdir(f"{VIDEO_DIR_PATH}/{dataset_folder_dropdown.value}")
    total_dataset_videos = len([file for file in dataset_folder_content if file.endswith('.mp4')])

    dataset_cutting_button.disabled = False
    dataset_cutting_button.description = f"Start Cutting ({total_dataset_videos})"

def video_cutting_select(change):
  os.makedirs(training_branch, exist_ok=True)
  training_dataset = f"{VIDEO_DIR_PATH}/{dataset_folder_dropdown.value}"

  # Loop video files from selected dataset folder
  for video_file in os.listdir(training_dataset):
    video, ext = os.path.splitext(video_file)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if ext != ".mp4":
      continue

    video_folder = f"{training_branch}/{video}"
    if not os.path.exists(video_folder):
      os.mkdir(video_folder)

      # Retrieve charade object by ID
      charade_data = None
      for charade in charades_all:
        if charade.id == video:
            charade_data = charade
            break

      # If no clipping required, keep whole video
      if not charade_data.actions:
        print(f"No clipping needed")
      else:
        print(f"Clipping {video}")
        charade_actions = charade_data.actions.items()
        total_charade_actions = len(charade_actions)
        for i, (class_id, timings) in enumerate(charade_actions):

          input_video = f"{training_dataset}/{video_file}"
          output_video = f"{video_folder}/{video}{i+1:02}{ext}"

          print(f"#{i+1}/{total_charade_actions}: {timings[0]}ms to {timings[0]+timings[1]}ms [I:{input_video}] [O:{output_video}]")
          !ffmpeg -i {input_video} -ss {timings[0]}ms -t {timings[1]}ms -c:v libx264 -c:a aac {output_video} -loglevel quiet
    else:
      print(f"Folder already exist for video_id: {video}. Skipping ...")
  print("Finished Clipping")

# Attach Listeners
dataset_folder_dropdown.observe(dataset_dir_select, 'value')
dataset_cutting_button.on_click(video_cutting_select)

# Display fields
display(dataset_folder_dropdown)
display(dataset_cutting_button)

Dropdown(description='Dataset Folder:', layout=Layout(width='300px'), options=('Skeleton',), style=Description…

Button(description='Start Cutting', disabled=True, layout=Layout(margin='0px 0px 20px 154px'), style=ButtonSty…

In [ ]:
# @title Dataset Preload (Metadata) { display-mode: "form" }

training_metadata_file = f"{training_branch}/metadata.tsv"
with open(training_metadata_file, 'w', newline='', encoding='utf-8') as tsvfile:
  fieldnames = ['part_id', 'clip_id', 'caption']
  writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
  writer.writeheader()

  for part_id in os.listdir(training_branch):
    folder_path = os.path.join(training_branch, part_id)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if not os.path.isdir(folder_path) or part_id.startswith("."):
      continue

    charade_data = None
    for charade in charades_all:
      if charade.id == part_id:
          charade_data = charade
          break

    if not charade_data:
      print("Missing charades data, skipping ...")
      continue

    # Sort by video sub-id to maintain order
    training_video_files = sorted(os.listdir(folder_path), key=lambda x: int(os.path.splitext(x)[0][-2:]))
    for i, clip in enumerate(training_video_files):
      caption = charade_data.getCaption(i)
      writer.writerow({
          'part_id': part_id,
          'clip_id': clip,
          'caption': caption
      })
print(f"TSV created: {training_metadata_file}")

FileNotFoundError: ignored

In [ ]:
# Changed fields in pose_train
## output_dir: "/content/custom_model/<model_name>" [test01]
## video_path: "/content/training_content/<dataset_folder>" [20231007_161145]

%cd {FYP_DIR_PATH}
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config="configs/pose_train.yaml"

In [ ]:
# @title Selection { display-mode: "form" }

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the base directory path
base_directory = '/content'

# Function to create nested dropdowns for directories
def create_directory_dropdown(directory_path, default_value='Select'):
    folder_picker = widgets.Dropdown(
        options=['Select'] + get_subdirectories(directory_path),
        value=default_value,
        description=os.path.basename(directory_path) + ':',
        disabled=False,
    )
    folder_picker.style.description_width = 'initial'  # Allow the description to shrink
    folder_picker.layout.width = 'auto'  # Allow the dropdown to adjust its width
    folder_picker.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    folder_picker.layout.width = '400px'
    return folder_picker

# Function to create "Training Epochs" textbox
def create_epochs_textbox(default_value=10):
    epochs_textbox = widgets.IntText(
        value=default_value,
        description='Training Epochs:',
        layout=widgets.Layout(width='auto')
    )
    epochs_textbox.style.description_width = 'initial'  # Allow the description to shrink
    epochs_textbox.layout.width = 'auto'  # Allow the textbox to adjust its width
    epochs_textbox.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    epochs_textbox.layout.width = '400px'
    return epochs_textbox

# Function to create "Batch Size" textbox
def create_batch_size_textbox(default_value=32):
    batch_size_textbox = widgets.IntText(
        value=default_value,
        description='Batch Size:',
        layout=widgets.Layout(width='auto')
    )
    batch_size_textbox.style.description_width = 'initial'  # Allow the description to shrink
    batch_size_textbox.layout.width = 'auto'  # Allow the textbox to adjust its width
    batch_size_textbox.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    batch_size_textbox.layout.width = '400px'
    return batch_size_textbox

# Function to create "Submit" button
def create_submit_button():
    submit_button = widgets.Button(
        description='Submit',
    )
    submit_button.layout.margin = '20px 0 0 200px'  # Add margin for positioning
    return submit_button

# Function to get subdirectories in a folder
def get_subdirectories(folder_path):
    folders = os.listdir(folder_path)
    return [content for content in folders if os.path.isdir(os.path.join(folder_path, content))]

# Function to display the selected values with full paths
def display_selected_values(selected_folder, subfolder, epochs, batch_size):
    clear_output(wait=True)
    dataset_folder_path = os.path.join(base_directory, selected_folder)
    subfolder_path = os.path.join(dataset_folder_path, subfolder) if subfolder else None
    print(f'Selected Dataset Folder: {dataset_folder_path}')
    if subfolder_path:
        print(f'Selected Dataset Subfolder: {subfolder_path}')
    print(f'Epochs Entered: {epochs}')
    print(f'Batch Size Entered: {batch_size}')

# Create an output widget as a placeholder
output_placeholder = widgets.Output()

# Create the "Dataset Folder" dropdown
dataset_folder_picker = create_directory_dropdown(base_directory)
dataset_folder_picker.description = 'Dataset Folder:'

# Create empty variables for the subfolder, epochs, and batch size widgets
subfolder_picker = None
epochs_textbox = None
batch_size_textbox = None
submit_button = None

# Function to handle button click
def on_submit_button_click(b):
    selected_folder = dataset_folder_picker.value
    subfolder = subfolder_picker.value if subfolder_picker else None
    epochs = epochs_textbox.value
    batch_size = batch_size_textbox.value
    display_selected_values(selected_folder, subfolder, epochs, batch_size)

# Function to update the directory dropdowns based on the selected "Dataset Folder"
def update_directory_dropdowns(change):
    global subfolder_picker, epochs_textbox, batch_size_textbox, submit_button

    selected_folder = change['new']

    # Clear previous subfolder, epochs, and batch size widgets
    if subfolder_picker:
        subfolder_picker.close()
    if epochs_textbox:
        epochs_textbox.close()
    if batch_size_textbox:
        batch_size_textbox.close()
    if submit_button:
        submit_button.close()

    # Create the "Dataset Subfolder" dropdown if a valid folder is selected
    if selected_folder != 'Select':
        subfolder_picker = create_directory_dropdown(os.path.join(base_directory, selected_folder), default_value='Select')
        subfolder_picker.description = 'Dataset Subfolder:'
        display(subfolder_picker)

        # Create "Training Epochs" and "Batch Size" textboxes
        epochs_textbox = create_epochs_textbox()
        batch_size_textbox = create_batch_size_textbox()
        submit_button = create_submit_button()
        submit_button.on_click(on_submit_button_click)

        # Display the "Submit" button
        display(epochs_textbox, batch_size_textbox, submit_button)

# Observe changes in the "Dataset Folder" dropdown
dataset_folder_picker.observe(update_directory_dropdowns, names='value')

# Display the widgets
display(dataset_folder_picker, output_placeholder)


# V2

## Classes

In [16]:
svg_loading = """<svg class="ring" viewBox="25 25 50 50" stroke-width="8"><circle cx="50" cy="50" r="20" /></svg>"""
css = """<style>.check {background-color: white;border-radius: 100%;fill: green;margin: 2.5px 6px 2.5px 0px;}.ring {--uib-size: 25px;--uib-speed: 2s;--uib-color: gray;height: var(--uib-size);width: var(--uib-size);vertical-align: middle;transform-origin: center;animation: rotate var(--uib-speed) linear infinite;margin: 2.5px 6px 2.5px 0px;}.ring circle {fill: none;stroke: var(--uib-color);stroke-dasharray: 1, 200;stroke-dashoffset: 0;stroke-linecap: round;animation: stretch calc(var(--uib-speed) * 0.75) ease-in-out infinite;}@keyframes rotate {100% {transform: rotate(360deg);}}@keyframes stretch {0% {stroke-dasharray: 1, 200;stroke-dashoffset: 0;}50% {stroke-dasharray: 90, 200;stroke-dashoffset: -35px;}100% {stroke-dashoffset: -124px;}}</style>"""
svg_done = """<svg width="25px" class="check" viewBox="0 0 16 16"><path stroke-width="10" d="M8 16A8 8 0 1 1 8 0a8 8 0 0 1 0 16Zm3.78-9.72a.751.751 0 0 0-.018-1.042.751.751 0 0 0-1.042-.018L6.75 9.19 5.28 7.72a.751.751 0 0 0-1.042.018.751.751 0 0 0-.018 1.042l2 2a.75.75 0 0 0 1.06 0Z" /></svg>"""


In [17]:
class State(Enum):
  INTRO = 1
  SETUP = 2
  EXPLORE = 3
  MMPOSE = 4
  FYP = 5

In [31]:
#@title Statemanager
class StateManager:
  def __init__(self):
    self.state = State.EXPLORE
    self.state_classes = {
        State.INTRO: IntroState,
        State.SETUP: SetupState,
        State.EXPLORE: ExplorationState,
        State.MMPOSE: MMPoseState,
        State.FYP: FYPState
    }
    # self.state_object = self.state_classes[self.state]
    self.screen = widgets.Output()

    self.state_object = self.state_classes[self.state](self)
    display(widgets.HTML(css))


  def start(self):
    display(self.screen)
    self.display_screen()

  def display_screen(self):
    with self.screen:
      clear_output()
      display(self.state_object.getScreen())

  def clear_console(self):
    clear_output(wait=True)
    display(widgets.HTML(css))
    display(self.screen)

  def change_state(self, new_state, content=None):
    self.state = new_state
    if content:
      self.state_object = self.state_classes[self.state](self, content=content)
    else:
      self.state_object = self.state_classes[self.state](self)
    self.clear_console()
    self.display_screen()

In [19]:
#@title BaseState
class BaseState:
  def __init__(self, state_manager, title):
    self.title = title
    self.state_manager = state_manager
  def getScreen(self):
    raise NotImplementedError()

In [20]:
#@title IntroState
class IntroState(BaseState):
  def __init__(self, state_manager):
    BaseState.__init__(self, state_manager, "Welcome")
    self.subtitle = widgets.HTML("""
      <b>A Jupyter Notebook Wrapper that combines the works of FollowYourPose & MMPose.</b><br>
      Brought to you by team 05:
      <ul>
        <li>Aloysius</li>
        <li>Max</li>
        <li>Zhenglong</li>
        <li>Ai Xin</li>
        <li>Huai Fu</li>
      </ul>
    """)
    self.get_started_button = widgets.Button(description="Get Started >>")
    self.get_started_button.on_click(lambda x: self.state_manager.change_state(State.SETUP))

    self.screen = widgets.AppLayout(
        header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
        left_sidebar=None,
        center=self.getContent(),
        right_sidebar=None,
        footer=None,
        pane_heights=[1,5,1]
    )

  def getScreen(self):
    return self.screen

  def getContent(self):
    vbox = widgets.VBox([self.subtitle, self.get_started_button])
    return vbox

In [21]:
#@title SetupState
class SetupState(BaseState):
  def __init__(self, state_manager):
    BaseState.__init__(self, state_manager, "Setup")
    self.platform = sys.platform

    ram_bytes = psutil.virtual_memory().available
    ram_gb = ram_bytes / (1024 ** 3)
    self.avail_ram = f"{ram_gb:.2f}"

    # Button
    self.skip_button = widgets.Button(description="Skip >>")
    self.skip_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.run_setup_button = widgets.Button(description="Setup", button_style="info")
    self.run_setup_button.on_click(lambda x: self.run_setup())

    self.mount_button = widgets.Button(description="Mount", button_style="info")
    self.mount_button.on_click(lambda x: self.run_mount_account_select())

    self.continue_button = widgets.Button(description="Continue", button_style="info")
    self.continue_button.on_click(lambda x: self.run_mount_drive())

    # Substates
    self.state = 0
    self.setup_stage = [False, False, False, False]

    self.screen = widgets.AppLayout(
        header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
        left_sidebar=None,
        center=None,
        right_sidebar=None,
        footer=None,
        pane_heights=[1,10,1]
      )

    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_platform = widgets.HTML(f"""
      <b>Current Platform: </b>{self.platform}<br>
      <b>Available Ram: </b>{self.avail_ram}GB<br>
      <b>Est Setup Time: </b>8 mins
    """)

    vbox_items = []
    if self.state == 0:
      hbox_button = widgets.HBox([self.run_setup_button, self.skip_button])
      vbox_items = [label_platform, hbox_button]
    elif self.state >= 1:
      hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))
      setup_label_1 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[0] else svg_loading),widgets.HTML("<b>Setting up python environment</b>")])
      setup_label_2 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[1] else svg_loading),widgets.HTML("<b>Cloning git repository</b>")])
      setup_label_3 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[2] else svg_loading),widgets.HTML("<b>Creating folder scaffolding</b>")])
      setup_label_4 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[3] else svg_loading),widgets.HTML("<b>Downloading dependency libraries</b>")])

      vbox_items = [label_platform, hr, setup_label_1, setup_label_2, setup_label_3, setup_label_4, hr]
      if self.state >= 2:
        label_mount = widgets.HTML("""
          <h2>Mount Drive</h2>
          <ol>
            <li>Click mount to start then wait for console output</li>
            <li>Click on the URL of the Failure in the output Failure("Error opening URL: ...")</li>
            <li>Login to the appropriate account</li>
            <li>Once successful, you can close the tab</li>
            <li><b>Click continue only when you have login into the correct account</b></li>
          </ol>
        """)
        vbox_items.append(label_mount)

        if self.state == 2:
          vbox_items.append(self.mount_button)
        if self.state == 3:
          vbox_items.append(self.continue_button)

    self.screen.center = widgets.VBox(vbox_items, layout=widgets.Layout(margin="0px 0px 30px 0px"))

  def run_setup(self):
    self.state = 1
    self.build_screen()

    setup_python()
    self.setup_stage[0] = True
    self.build_screen()
    setup_github()
    self.setup_stage[1] = True
    self.build_screen()
    setup_folder_scaffolding()
    self.setup_stage[2] = True
    self.build_screen()
    setup_dependency_lib()
    self.setup_stage[3] = True

    self.state = 2
    self.build_screen()

  def run_mount_account_select(self):
    self.state_manager.clear_console()
    self.build_screen()

    mount_account_select()
    self.state = 3
    self.build_screen()

  def run_mount_drive(self):
    mount_drive()
    create_dir_path()
    symbolic_linking()
    self.state_manager.change_state(State.EXPLORE)


In [22]:
#@title ExplorationState

class ExplorationState(BaseState):
  def __init__(self, state_manager):
    BaseState.__init__(self, state_manager, "Data Exploration")

    self.video_output = widgets.Output()
    self.video_tab = VideoTab(self)

    self.video_tab.attach_listeners()
    self.video_tab.display_content()

    tab = widgets.Tab(layout=widgets.Layout(width="900px"))
    tab.children = [self.video_output]
    tab.set_title(0, "Video")

    self.screen = widgets.AppLayout(
        header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
        left_sidebar=None,
        center=tab,
        right_sidebar=None,
        footer=widgets.HTML("<style>.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tabLabel{color: white; font-weight: bold;}</style>"),
        pane_heights=[1,10,1]
    )

  def getScreen(self):
    return self.screen

  def getContent(self):
    vbox = widgets.HTML(value=f"<h1>{self.title}</h1>") # widgets.VBox([self.subtitle, self.get_started_button])
    return vbox

  def change_state(self, new_state, content):
    self.state_manager.change_state(new_state, content)


In [23]:
#@title MMPoseState

class MMPoseState(BaseState):
  def __init__(self, state_manager, content):
    BaseState.__init__(self, state_manager, "MMPose")
    self.content = content
    self.screen = widgets.AppLayout(
      header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
      left_sidebar=None,
      center=None,
      right_sidebar=None,
      footer=None,
      pane_heights=[1,5,1]
    )
    self.state = 1
    self.mmpose_stage = [False, False]

    self.back_button = widgets.Button(description="Back")
    self.back_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.start_button = widgets.Button(description="Start", button_style="info")
    self.start_button.on_click(lambda x: self.run_mmpose())

    self.done_button = widgets.Button(description="Done", button_style="info")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_video = widgets.HTML(f"""
      <b>Selected Video: </b>{self.content[1]}<br>
    """)

    vbox_content = []
    if self.state == 1:
      hbox_button = widgets.HBox([self.start_button, self.back_button])
      vbox_content = [label_video, hbox_button]
    elif self.state >= 2:
      hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))
      mmpose_label_1 = widgets.HBox([widgets.HTML(svg_done if self.mmpose_stage[0] else svg_loading),widgets.HTML("<b>Converting video to skeleton</b>")])
      mmpose_label_2 = widgets.HBox([widgets.HTML(svg_done if self.mmpose_stage[1] else svg_loading),widgets.HTML("<b>Re-encoding video</b>")])

      vbox_content = [label_video, hr, mmpose_label_1, mmpose_label_2, hr]

      if self.state == 3:
        vbox_content.append(self.done_button)

    self.screen.center = widgets.VBox(vbox_content)

  def run_mmpose(self):
    self.state = 2
    self.build_screen()

    folder_path, video_path = self.content

    run_mmpose_inference(folder_path, video_path)
    self.mmpose_stage[0] = True
    self.build_screen()

    reencode_video(video_path)
    self.mmpose_stage[1] = True
    self.build_screen()

    self.state = 3
    self.build_screen()




## WIP

In [76]:
#@title FYPState

class FYPState(BaseState):
  def __init__(self, state_manager, content):
    BaseState.__init__(self, state_manager, "FollowYourPose")
    self.content = content
    self.screen = widgets.AppLayout(
      header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
      left_sidebar=None,
      center=None,
      right_sidebar=None,
      footer=None,
      pane_heights=[1,10,1]
    )
    self.stage = 1
    self.fyp_stage = [False, False]

    self.start_button = widgets.Button(description="Start", button_style="info")
    self.start_button.on_click(lambda x: self.run_fyp())

    self.done_button = widgets.Button(description="Done", button_style="info")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.FYPConfig = FYPConfig()
    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_video = widgets.HTML(f"""
      <b>Selected Skeleton Video: </b>{self.content[1]}<br>
    """)
    hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))

    vbox_content = []
    if self.stage == 1:
      vbox_content = [label_video, self.FYPConfig.getForm(), hr, self.start_button]
    elif self.stage >= 2:
      fyp_label_1 = widgets.HBox([widgets.HTML(svg_done if self.fyp_stage[0] else svg_loading),widgets.HTML("<b>Saving configs</b>")])
      fyp_label_2 = widgets.HBox([widgets.HTML(svg_done if self.fyp_stage[1] else svg_loading),widgets.HTML("<b>Running inference</b>")])

      vbox_content = [label_video, hr, fyp_label_1, fyp_label_2, hr]

      if self.stage == 3:
        label_folder_output = widgets.HTML(f"<b>Output Folder: </b>{self.FYPConfig.output_dir.value}<br>")

        vbox_content.extend([label_folder_output, self.done_button])

    self.screen.center = widgets.VBox(vbox_content)

  def run_fyp(self):
    self.stage = 2
    self.build_screen()

    self.FYPConfig.save_config()
    self.fyp_stage[0] = True
    self.build_screen()

    run_fyp_inference(f"{CONFIG_DIR_PATH}/pose_sample.yaml", f"{VIDEO_DIR_PATH}/{self.content[0]}/{self.content[1]}")
    self.fyp_stage[1] = True
    self.build_screen()

    self.stage = 3
    self.build_screen()


In [77]:
#@title App
state_manager = StateManager()
state_manager.start()

HTML(value='<style>.check {background-color: white;border-radius: 100%;fill: green;margin: 2.5px 6px 2.5px 0px…

Output()